In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn

In [2]:
read_data = pd.read_csv('training.csv')
read_test = pd.read_csv('test.csv')
column_details = pd.read_csv('Xente_Variable_Definitions.csv')

In [3]:
read_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [4]:
read_data.shape

(95662, 16)

In [5]:
column_details

,Column Name,Definition
0,TransactionId,Unique �transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transacti...
2,AccountId,Unique number identifying the customer on plat...
3,SubscriptionId,Unique number identifying the customer subscri...
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item �bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader pr...


In [6]:
read_data['BatchId'][:4]

0     BatchId_36123
1     BatchId_15642
2     BatchId_53941
3    BatchId_102363
Name: BatchId, dtype: object

In [7]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
TransactionId           95662 non-null object
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null object
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: float64(1), int64(4), object(11)
memory usage: 11.7+ MB


# Taking a look at transaction ID

In [8]:
read_data['TransactionId'][:10]

0     TransactionId_76871
1     TransactionId_73770
2     TransactionId_26203
3       TransactionId_380
4     TransactionId_28195
5     TransactionId_23223
6    TransactionId_118063
7    TransactionId_100640
8     TransactionId_51905
9    TransactionId_130161
Name: TransactionId, dtype: object

In [9]:
np.unique(read_data['TransactionId']).shape

(95662,)

# BatchID

In [10]:
read_data['BatchId'][:10]

0     BatchId_36123
1     BatchId_15642
2     BatchId_53941
3    BatchId_102363
4     BatchId_38780
5     BatchId_25954
6    BatchId_118460
7     BatchId_38561
8     BatchId_93774
9     BatchId_82409
Name: BatchId, dtype: object

In [11]:
dict_batch_id = read_data['BatchId'].value_counts()


In [12]:
ids = read_data[['TransactionId','BatchId','AccountId', 'SubscriptionId','CustomerId', 'ProviderId', 'ProductId','ChannelId']]

In [13]:
train_data = read_data.drop(['TransactionId','BatchId','AccountId', 'SubscriptionId','CustomerId', 'ProviderId', 'ProductId','ChannelId'], axis = 1)

In [14]:
train_data.head()

,CurrencyCode,CountryCode,ProductCategory,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,UGX,256,airtime,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,UGX,256,financial_services,-20.0,20,2018-11-15T02:19:08Z,2,0
2,UGX,256,airtime,500.0,500,2018-11-15T02:44:21Z,2,0
3,UGX,256,utility_bill,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,UGX,256,financial_services,-644.0,644,2018-11-15T03:34:21Z,2,0


In [15]:
np.unique(train_data['CurrencyCode'])

array(['UGX'], dtype=object)

In [16]:
train_data = train_data.drop('CurrencyCode', axis = 1)

In [17]:
train_data.head()

,CountryCode,ProductCategory,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,256,airtime,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,256,financial_services,-20.0,20,2018-11-15T02:19:08Z,2,0
2,256,airtime,500.0,500,2018-11-15T02:44:21Z,2,0
3,256,utility_bill,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,256,financial_services,-644.0,644,2018-11-15T03:34:21Z,2,0


In [18]:
np.unique(train_data['CountryCode'])

array([256], dtype=int64)

In [19]:
train_data = train_data.drop('CountryCode', axis = 1)

In [20]:
train_data.head()

,ProductCategory,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,airtime,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,financial_services,-20.0,20,2018-11-15T02:19:08Z,2,0
2,airtime,500.0,500,2018-11-15T02:44:21Z,2,0
3,utility_bill,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,financial_services,-644.0,644,2018-11-15T03:34:21Z,2,0


In [21]:
np.unique(train_data['ProductCategory'])

array(['airtime', 'data_bundles', 'financial_services', 'movies', 'other',
       'ticket', 'transport', 'tv', 'utility_bill'], dtype=object)

In [22]:
np.unique(train_data['PricingStrategy'])

array([0, 1, 2, 4], dtype=int64)

In [23]:
data = pd.get_dummies(train_data, columns = ['ProductCategory'])
data.shape

(95662, 14)

In [24]:
data.head()

,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,1000.0,1000,2018-11-15T02:18:49Z,2,0,1,0,0,0,0,0,0,0,0
1,-20.0,20,2018-11-15T02:19:08Z,2,0,0,0,1,0,0,0,0,0,0
2,500.0,500,2018-11-15T02:44:21Z,2,0,1,0,0,0,0,0,0,0,0
3,20000.0,21800,2018-11-15T03:32:55Z,2,0,0,0,0,0,0,0,0,0,1
4,-644.0,644,2018-11-15T03:34:21Z,2,0,0,0,1,0,0,0,0,0,0


In [25]:
transaction = data['TransactionStartTime']
val = transaction[0]
val.split(':')[0].split('-')

['2018', '11', '15T02']

In [26]:
def trans(data):
    year, month = [],[]
    vals = []
    for k in data:
        vals = k.split(':')[0].split('-')
        year.append(int(vals[0]))
        month.append(int(vals[1]))
    return year, month

year, month = trans(transaction)
len(year), len(month)
        

(95662, 95662)